# Clustering the swiss map

Our idea is to create squares all over the swiss map to create areas and then count the accident per area

Swiss coordinates system is designed so that no two x and y coordinates can be mixed, 1 unit is one meter on the field.

min coord Y : 480000, 070000 (bottom left)
max coord X : 840000, 300000 (top right)

There are squares that are outside Switzerland, but we are not interesting in area with 0 value (as if there are not road there cannot be any accident as well)

In [ ]:
import math
import numpy as np
xRange=840000-480000
yRange=300000-70000
print(xRange, yRange)

In [ ]:
#PAS TRES UTILE, IDEE DE DEPART
print("very small", "0.5 x 0.5\t", xRange/500, "x", yRange/500, "\t", math.ceil(xRange/500 * yRange/500), "squares") #0.25 km2
print("small","1 x 1\t\t", xRange/1000,"x", yRange/1000, "\t", math.ceil(xRange/1000 * yRange/1000), "squares")       #1 km2
print("medium", "1.5 x 1.5\t", xRange/1500,"x", yRange/1500, "\t", math.ceil(xRange/1500 * yRange/1500), "squares")   #2.25 km2
print("large", "2 x 2\t\t", xRange/2000,"x", yRange/2000, "\t", math.ceil(xRange/2000 * yRange/2000), "squares")      #4 km2
print("very large", "5 x 5\t", xRange/5000,"x", yRange/5000, "\t", math.ceil(xRange/5000 * yRange/5000), "squares") #25 km2
print("huge", "10 x 10\t\t", xRange/10000,"x", yRange/10000, "\t", math.ceil(xRange/10000 * yRange/10000), "squares") #100 km2

In [ ]:
def create_count_grid(square_size, df):
    """
    Creates and fills an array with the count of accidents per squares of size 'square_size' on the map with the provided DataFrame
    """
    nb_x=math.ceil(xRange/square_size)
    nb_y=math.ceil(yRange/square_size)
    
    grid=np.ndarray((nb_x, nb_y))
    grid.fill(0)
    
    for index, row in df.iterrows():
        x=math.floor(row['geometry_coordinates'][0]/nb_x)
        y=math.floor(row['geometry_coordinates'][1]/nb_y)
        grid[x,y]+=1
    return grid

In [ ]:
def compute_ratio(grid):
    """
    To find a good square size length, we compute a ratio that is (% of non empty squares)/(mean of non zero values)
    """
    non_empty_indices=np.nonzero(grid)
    grid_non_zero=grid[non_empty_indices]
    
    non_empty_pct=len(non_empty_indices)/(grid.shape[0]*grid.shape[1])
    
    mean=grid.mean
    mean_non_zero=grid_non_zero.mean
    
    #return non_empty_pct/mean
    return non_empty_pct/mean_non_zero   

In [ ]:
def optimality_plot(df):
    """
    Plot the result of the ratio per square length
    """
    grid_ratio_list=list()
    for i in range (500, 15000, 500):
        tmp_grid=create_count_grid(i, df)
        grid_ratio_list.append(i,compute_ratio(tmp_grid))     